In [1]:
# -*- coding: utf-8 -*-
import sys
import os
import numpy as np
import pandas as pd
import cobra
import matplotlib.pyplot as plt 
plt.rc('axes', axisbelow=True)

print('Python version:', sys.version)
print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('cobrapy version:', cobra.__version__)

Python version: 3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.18.3
pandas version: 1.0.3
cobrapy version: 0.15.4


In [2]:
def KORxn(model, rxns2KO):
    """Function for knocking out reactions."""
    for ID in rxns2KO:
        reaction = model.reactions.get_by_id(ID)
        reaction.knock_out()


In [3]:
def flux2file(model, product, psw, output_dir='tmp'):
    """Function of exporting flux data."""
    n = len(model.reactions)
    modelMatrix = np.empty([n, 9], dtype = object)
    for i in range(len(model.reactions)):
        x = model.reactions[i]
        modelMatrix[i, 0] = i + 1
        modelMatrix[i, 1] = x.id
        modelMatrix[i, 2] = x.name
        modelMatrix[i, 3] = x.reaction
        modelMatrix[i, 4] = x.subsystem
        modelMatrix[i, 5] = x.lower_bound
        modelMatrix[i, 6] = x.upper_bound
        modelMatrix[i, 7] = x.flux
        modelMatrix[i, 8] = abs(x.flux)
        
    df = pd.DataFrame(data = modelMatrix, 
                        columns = ['N', 'RxnID', 'RxnName', 'Reaction', 'SubSystem', 
                        'LowerBound', 'UpperBound', 'Flux-core', 'abs(Flux)'])
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    filepath = os.path.join(output_dir, '{}_{}.xlsx'.format(product, psw))
    df.to_excel(filepath, index=False)
    

#### _E. coli_ core model with some changes:	

* Corrected the transhydrogenase (THD2) to one proton translocation;
* Deactivated pyruvate formate lyase (PFL) under aerobic coditions.

In [4]:
model = cobra.io.load_json_model(r'..\0_ecoli_models\e_coli_core.json')

model.reactions.THD2.subtract_metabolites({"h_e": -1, "h_c": 1})

In [5]:
KORxn_base = ['PFL']
KORxn(model, KORxn_base)

In [6]:
model.reactions.EX_glc__D_e.bounds = (0,0)

model_wt = model.copy()

### Model curations: 
Anticipated results of a model knocked out the _lpd_ gene:  
* no growth on acetate only;  
* grow on acetate + formate (with formate dehydrogenase), and where formate serves only as electron source

In [7]:
# knock out lpdA
model.genes.b0116.knock_out()

In [8]:
# on acetate only
with model as m: 
    m.reactions.EX_ac_e.lower_bound = -10  # arbitrary 
    cobra.flux_analysis.pfba(m)
    m.summary()
    flux2file(m,'biomass_1','lpd_core')


IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ----------------------
o2_e   15      h2o_e  15.9   BIOMASS_Ecol...  0.115
ac_e   10      co2_e  15.1
h_e     7.69
nh4_e   0.627
pi_e    0.423


The model uses the oxidative pentose phosphate pathway generating NADPH, 
which is then used by NAD transhydrogenase (NADTRHD, sthA) transferring the electrons.  

Try to block it by KO GND.

In [9]:
with model_wt as m:
    m.reactions.EX_ac_e.lower_bound = -10  
    cobra.flux_analysis.pfba(m)
    m.summary()
    
    print('\n','* - * - *')
    m.reactions.GND.knock_out()
    cobra.flux_analysis.pfba(m)
    m.summary()
    

IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ----------------------
o2_e   12.4    h2o_e  13.9   BIOMASS_Ecol...  0.173
ac_e   10      co2_e  12.6
h_e     6.52
nh4_e   0.945
pi_e    0.638

 * - * - *
IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ----------------------
o2_e   12.4    h2o_e  13.9   BIOMASS_Ecol...  0.173
ac_e   10      co2_e  12.6
h_e     6.52
nh4_e   0.945
pi_e    0.638


In [ ]:
# KO GND doesn't change the results on WT, so block it in the model.
with model as m: 
    m.reactions.EX_ac_e.lower_bound = -10
    KORxn(m, ['GND'])
    cobra.flux_analysis.pfba(m)
    m.summary()
    flux2file(m,'biomass_2','lpd_core')

# becoming infeasible

In [10]:
# not growing on acetate now, try to add formate
# adding FDH reaction to the model
model_FDH = model.copy()
FDH = cobra.Reaction(id="FDH", name="formate dehydrogenase", lower_bound=0, upper_bound=1000) 
model_FDH.add_reaction(FDH)
FDH.add_metabolites({"for_c": -1, "nad_c": -1, "co2_c": 1, "nadh_c": 1})

with model_FDH as m: 
    m.reactions.EX_ac_e.lower_bound = -10
    m.reactions.EX_for_e.lower_bound = -100
    KORxn(m, ['GND'])
    cobra.flux_analysis.pfba(m)
    m.summary()
    flux2file(m,'biomass_for_1','lpd_core')
    print('\n','* - * - *')
    m.metabolites.for_c.summary()

IN FLUXES     OUT FLUXES    OBJECTIVES
------------  ------------  ----------------------
h_e    37.6   h2o_e  41.6   BIOMASS_Ecol...  0.391
for_e  35.4   co2_e  38.8
o2_e   20.6
ac_e   10
nh4_e   2.13
pi_e    1.44

 * - * - *
PRODUCING REACTIONS -- Formate (for_c)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------
100%    35.4  FORt2     for_e + h_e --> for_c + h_c

CONSUMING REACTIONS -- Formate (for_c)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------
100%    35.4  FDH       for_c + nad_c --> co2_c + nadh_c


The `model` now cannot use acetate as a sole carbon source.  
Formate (with FDH) can be used as electron source. And it serves only as electron donor. 


In [11]:
KORxn(model_wt,['GND'])
KORxn(model, ['GND'])
KORxn(model_FDH,['GND'])

### Calculating the substrate uptake rates: 
WT grown on aceate has doubling time **2.9 h**, i.e., **0.24 h<sup>-1</sup>**.

In [12]:
DT = 2.9   # WT grow on 20 mM acetate -> DT = 2.9 h
gr = np.log(2) / DT  
print('growth rate is %.2f / hr' % gr)

growth rate is 0.24 / hr


In [13]:
with model_wt as m:
    m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
    m.reactions.EX_glc__D_e.bounds = (0,0)
    m.reactions.EX_ac_e.bounds = (-1000,0)
    m.objective = {m.reactions.EX_ac_e: -1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()

IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ------------
o2_e   15.6    h2o_e  17.7   EX_ac_e  13
ac_e   13      co2_e  15.9
h_e     8.25
nh4_e   1.3
pi_e    0.879


In [14]:
# lpd on glucose + acetate
with model as m:
    m.reactions.EX_glc__D_e.lower_bound = -10.5
    m.reactions.EX_ac_e.lower_bound = -13
    cobra.flux_analysis.pfba(m)
    m.summary()

    sol = m.objective.value   # 1/hour

print('\n', 'The doubling time is %.2f hr' % (np.log(2) / sol))

IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ----------------------
o2_e      15     h2o_e  29.7   BIOMASS_Ecol...  0.445
ac_e      13     pyr_e  21.8
glc__D_e  10.5   h_e    17.7
nh4_e      2.43  co2_e   4.63
pi_e       1.64

 The doubling time is 1.56 hr


The maximum acetate uptake rate is **13.0 mmol/gCDW/h**, which is within the range of 11 - 15.5, reported in https://doi.org/10.1038/84379.   

Also, on glc + Ac, the doubling time is colse to the experiment value *1.6 h* (Fig. 2). Here the glc uptake rate is set to 10.5 according to [Varma & Palsson, 1994](https://pubmed.ncbi.nlm.nih.gov/7986045/).

Therefore, the model is good: _lpd_ (b0116, affecting PDH & AKGDH), GND and PFL are knocked out.

The yield can be calculated as the ratio between the growth rate and the substrate uptake/utilization rate:   
$$\dfrac{1/h}{mmol/gCDW/h} = \dfrac{gCDW}{mmol} = \dfrac{gCDW/L}{mmol/L}$$

Using BionNumber [109836](https://bionumbers.hms.harvard.edu/bionumber.aspx?s=n&v=4&id=109836), one unit of OD600 _E. coli_ corresponds to a cell dry weight of _0.39 gCDW/L_, biomass yield can be converted to OD600 (increase).  

**Assuming all the limiting substrates are consumed in the experiments** 

In [15]:
OD2Y = 0.39  # gCDW/L per one unit OD600 culture

### _$\Delta$lpd_ growing on acetate + formate:  
For the _$\Delta$lpd_ strain grows on 20 mM acetate and formate:  

The growth rates of the $\Delta$_lpd_ strain on acetate and formate are lower than WT on acetate. Especially when formate concentration below 13 mM, the growth rate increased along with the formate concentration increase, this could indicate that formate uptake (rate) or formate oxidation is the limit under low concentrations.  

Also, the maximum ODs are all below the WT on acetate alone. The maximum ODs increased along with the increase of formate concentration. These indicates that acetate is not the limit but formate (energy source). For the $\Delta$_lpd_ strain on acetate and formate, formate is the major energy source, acetate serves mainly as carbon source, the strain should have higher biomass yield than WT on soly acetate, where acetate is both carbon and energy sources.

In [16]:
# Block all exchange reactions to avoid secretion
for rxn in model_FDH.exchanges:
    if "C" in rxn.check_mass_balance(): # knock out all other carbon-related transporters
        rxn.bounds = (0, 0)
model_FDH.reactions.EX_co2_e.bounds = (-1000, 1000)

with model_FDH as m: 
    gr = np.log(2) / 5.2  # 20 mM ac + 44 mM formate, the highest growth rate
    m.reactions.EX_ac_e.lower_bound = -13
    m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
    m.reactions.EX_for_e.lower_bound = -1000
    m.objective = {m.reactions.EX_for_e: -1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()
    for_up = m.objective.value
    flux2file(m,'biomass_for_4','lpd_core')

print('\n', '* - * - *')
print(f'Yield is {gr/for_up/OD2Y: 0.1e} unit OD600 per mM formate')

IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  --------------
h_e    17.2    h2o_e  18.6   EX_for_e  16.5
for_e  16.5    co2_e  17.6
o2_e    9.23
ac_e    3.41
nh4_e   0.727
pi_e    0.49

 * - * - *
Yield is  2.1e-02 unit OD600 per mM formate


Formate is the limiting as shown in the experimental results.  

Here acetate uptake rate is _3.41 mmol/gCDW/h_, (the maximum) formate uptake rate (or FDH) is calculated to be **16.5 mmol/gCDW/h**. 
* The ratio of formate/acetate is **4.8**.  
* Biomass yield is **$ 2 \times{10 ^ {-2}}$ unit OD600 per mM formate**.

In [17]:
growth_for = pd.DataFrame({
    'formate_conc': np.logspace(0,5,6,base=1/1.5)*100,
    'max_OD600': np.array([0.43,0.363,0.273,0.211,0.163,0.122]),
    'doubling_time': np.array([6.1,5.6,5.2,5.2,5.5,5.8])
})

growth_for['Yield'] = growth_for['max_OD600'] / growth_for['formate_conc']    # unit OD600 per mM formate
growth_for['growth_rate'] = np.log(2) / growth_for['doubling_time']
growth_for

,formate_conc,max_OD600,doubling_time,Yield,growth_rate
0,100.000000,0.430,6.1,0.004300,0.113631
1,66.666667,0.363,5.6,0.005445,0.123776
2,44.444444,0.273,5.2,0.006143,0.133298
3,29.629630,0.211,5.2,0.007121,0.133298
4,19.753086,0.163,5.5,0.008252,0.126027
5,13.168724,0.122,5.8,0.009264,0.119508


Since formate (uptake or oxidation) is the limit for growth, it determines the growth rate.

In [18]:
with model_FDH as m: 
    m.reactions.EX_ac_e.lower_bound = -13  # uptake rate, in mmol/gCDW/h
    m.reactions.EX_for_e.lower_bound = -1000
    m.objective = {m.reactions.EX_for_e: -1}
    m.objective_direction = 'min'
    for i in range(len(growth_for['growth_rate'])):
        gr = growth_for.loc[i, 'growth_rate']
        m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
        cobra.flux_analysis.pfba(m)
        growth_for.loc[i, 'EX_formate'] = abs(m.reactions.EX_for_e.flux)
        growth_for.loc[i, 'EX_acetate'] = abs(m.reactions.EX_ac_e.flux)

growth_for['Yield_cal'] = growth_for['growth_rate']/growth_for['EX_formate'] / OD2Y  # unit OD600 per mM formate
growth_for['for/ac'] = growth_for['EX_formate'] / growth_for['EX_acetate']

growth_for

,formate_conc,max_OD600,doubling_time,Yield,growth_rate,EX_formate,EX_acetate,Yield_cal,for/ac
0,100.000000,0.430,6.1,0.004300,0.113631,15.046770,2.905571,0.019364,5.178594
1,66.666667,0.363,5.6,0.005445,0.123776,15.790946,3.164997,0.020099,4.989246
2,44.444444,0.273,5.2,0.006143,0.133298,16.489326,3.408458,0.020728,4.837767
3,29.629630,0.211,5.2,0.007121,0.133298,16.489326,3.408458,0.020728,4.837767
4,19.753086,0.163,5.5,0.008252,0.126027,15.956017,3.222542,0.020252,4.951376
5,13.168724,0.122,5.8,0.009264,0.119508,15.477879,3.055859,0.019798,5.064985


### _$\Delta$lpd_ growing on acetate + methanol:  


In [19]:
# building the model
model_MDH = model.copy()

meoh_c = cobra.Metabolite(id='meoh_c',
                          formula='CH4O',
                          name='methanol',
                          compartment='c')
fald_c = cobra.Metabolite(id='fald_c',
                          formula='CH2O',
                          name='Formaldehyde',
                          compartment='c')

EX_meoh_e = cobra.Reaction(id="EX_meoh_e", name="methanol exchange", lower_bound=0, upper_bound=1000) 
model_MDH.add_reaction(EX_meoh_e)
EX_meoh_e.add_metabolites({meoh_c: -1})

MeDH = cobra.Reaction(id="MeDH", name="methanol dehydrogenase", lower_bound=0, upper_bound=1000) 
model_MDH.add_reaction(MeDH)
MeDH.add_metabolites({meoh_c: -1, "nad_c": -1, fald_c: 1, "nadh_c": 1, 'h_c': 1})

FRM = cobra.Reaction(id="FRM", name="formaldehyde oxidation, overall", lower_bound=0, upper_bound=1000) 
model_MDH.add_reaction(FRM)
FRM.add_metabolites({fald_c: -1, "nad_c": -1, 'h2o_c': -1, "for_c": 1, "nadh_c": 1, 'h_c': 2})

In [20]:
# Block exchange reactions to avoid secretion, expect formate
for rxn in model_MDH.exchanges:
    if "C" in rxn.check_mass_balance(): # knock out all other carbon-related transporters
        rxn.bounds = (0, 0)
model_MDH.reactions.EX_co2_e.bounds = (-1000, 1000)
model_MDH.reactions.EX_for_e.bounds = (0,1000)  # no FDH overexpression

with model_MDH as m:
    gr = np.log(2) / 32  # 20 mM ac + 667 mM MeOH, the highest growth rate
    m.reactions.EX_ac_e.lower_bound = -13
    m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.objective = {m.reactions.EX_meoh_e: -1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()
    flux2file(m,'biomass_meoh_2','lpd_core')


IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ---------------
o2_e   4.77    h2o_e  4.96   EX_meoh_e  4.61
ac_e   0.554   for_e  4.61
nh4_e  0.118   h_e    4.49
pi_e   0.0797  co2_e  0.186


Assuming the acetate uptake rate to be the maximum as WT on acetate, _13 mmol/gCDW/h_, the maximum MeOH uptake is calculated to be **4.61 mmol/gCDW/h**. Similar to the formate. 


In [21]:
growth_meoh = pd.DataFrame({
    'MeOH_conc': np.logspace(0,3,4,base=1/1.5)*1000,
    'max_OD600': np.array([0.278,0.504,0.341,0.234]),
    'doubling_time': np.array([38,32,37,37])    
})

growth_meoh['Yield'] = growth_meoh['max_OD600'] / growth_meoh['MeOH_conc']  # unit OD600 per mM methanol
growth_meoh['growth_rate'] = np.log(2) / growth_meoh['doubling_time']

with model_MDH as m:
    m.reactions.EX_ac_e.lower_bound = -13
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.objective = {m.reactions.EX_meoh_e: -1}
    m.objective_direction = 'min'
   
    for i in range(len(growth_meoh['growth_rate'])):
        # gr = growth_meoh.loc[i, 'growth_rate']
        gr = growth_for.loc[i,'growth_rate']
        m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
        cobra.flux_analysis.pfba(m)
        flux2file(m,f'meoh_{i}','lpd_for')
        growth_meoh.loc[i, 'EX_meoh'] = abs(m.reactions.EX_meoh_e.flux)
        growth_meoh.loc[i, 'EX_acetate'] = abs(m.reactions.EX_ac_e.flux)

growth_meoh['Yield_cal'] = growth_meoh['growth_rate'] / growth_meoh['EX_meoh'] / OD2Y  # unit OD600 per mM methanol
growth_meoh['meoh/ac'] = growth_meoh['EX_meoh'] / growth_meoh['EX_acetate']

growth_meoh

,MeOH_conc,max_OD600,doubling_time,Yield,growth_rate,EX_meoh,EX_acetate,Yield_cal,meoh/ac
0,1000.000000,0.278,38,0.000278,0.018241,8.359317,2.905571,0.005595,2.876996
1,666.666667,0.504,32,0.000756,0.021661,8.772748,3.164997,0.006331,2.771803
2,444.444444,0.341,37,0.000767,0.018734,9.160737,3.408458,0.005244,2.687648
3,296.296296,0.234,37,0.000790,0.018734,9.160737,3.408458,0.005244,2.687648


### The gC<sub>1</sub>M-gC<sub>2</sub>M strain growing on glucose + methanol  

In [22]:
# growth data without formate
growth_MDH = pd.DataFrame({
    'MDH':['CnMDH', 'BmMDH*','BsMDH','CgMDH'],
    'doubling_time': [2.7,3,3,10]
})
growth_MDH['growth_rate'] = np.log(2) / growth_MDH['doubling_time']
growth_MDH

,MDH,doubling_time,growth_rate
0,CnMDH,2.7,0.256721
1,BmMDH*,3.0,0.231049
2,BsMDH,3.0,0.231049
3,CgMDH,10.0,0.069315


In [23]:
# growth data with foramte
growth_MDH_for = pd.DataFrame({
    'MDH':['CnMDH', 'BmMDH*','BsMDH','CgMDH'],
    'doubling_time': [2.2,1.7,2.1,1.6]
})
growth_MDH_for['growth_rate'] = np.log(2) / growth_MDH_for['doubling_time']  # 1/h
growth_MDH_for

,MDH,doubling_time,growth_rate
0,CnMDH,2.2,0.315067
1,BmMDH*,1.7,0.407734
2,BsMDH,2.1,0.330070
3,CgMDH,1.6,0.433217


In [24]:
# build the model
model_C1 = model_wt.copy()
full_model = cobra.io.load_json_model(r'..\0_ecoli_models\iML1515.json')

newRxns = [
    model_MDH.reactions.EX_meoh_e,
    model_MDH.reactions.MeDH,
    model_MDH.reactions.FRM,
    full_model.reactions.FTHFLi,  # formate-THF ligase
    full_model.reactions.MTHFC,   # Methenyltetrahydrofolate cyclohydrolase
    full_model.reactions.MTHFD,   # Methylenetetrahydrofolate dehydrogenase (NADP)
    full_model.reactions.GLYCL,   # GCV
    full_model.reactions.GHMT2r   # GlyA
]

# to check the reactions
for i in newRxns:
    print(i)

EX_meoh_e: meoh_c --> 
MeDH: meoh_c + nad_c --> fald_c + h_c + nadh_c
FRM: fald_c + h2o_c + nad_c --> for_c + 2 h_c + nadh_c
FTHFLi: atp_c + for_c + thf_c --> 10fthf_c + adp_c + pi_c
MTHFC: h2o_c + methf_c <=> 10fthf_c + h_c
MTHFD: mlthf_c + nadp_c <=> methf_c + nadph_c
GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c
GHMT2r: ser__L_c + thf_c <=> gly_c + h2o_c + mlthf_c


In [25]:
newRxns[6].bounds = (-1000,1000)  # change GCV to be reversible
model_C1.add_reactions(newRxns)   # add the new reactions to the model

# update the biomass function by replacing 3pg to ser, gly and C1
# from DOI 10.1021/acssynbio.8b00093
model_C1.reactions.BIOMASS_Ecoli_core_w_GAM.build_reaction_from_string("0.462 ser__L_c + 1.033 gly_c + 1.033 mlthf_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 3.4454 glu__L_c + 59.35 h2o_c + 2.051 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c -> 59.81 adp_c + 2.6222 akg_c + 3.7478 coa_c + 58.27 h_c + 2.051 nadh_c + 13.0279 nadp_c + 58.31 pi_c + 1.033 thf_c")
print(model_C1.reactions.BIOMASS_Ecoli_core_w_GAM)

BIOMASS_Ecoli_core_w_GAM: 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 3.4454 glu__L_c + 1.033 gly_c + 59.35 h2o_c + 1.033 mlthf_c + 2.051 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c + 0.462 ser__L_c --> 59.81 adp_c + 2.6222 akg_c + 3.7478 coa_c + 58.27 h_c + 2.051 nadh_c + 13.0279 nadp_c + 58.31 pi_c + 1.033 thf_c


In [26]:
# assuming glc uptake rate is -10.5 as WT
with model_C1 as m:
    m.reactions.EX_glc__D_e.lower_bound = -10.5
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.objective = 'MeDH'
    m.objective_direction = 'min'
    for i in range(len(growth_MDH)):
        gr = growth_MDH.loc[i,'growth_rate']
        m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
        cobra.flux_analysis.pfba(m)
        growth_MDH.loc[i,'MeDH_flux'] = m.reactions.MeDH.flux  # in mmol/gCDW/h

growth_MDH

,MDH,doubling_time,growth_rate,MeDH_flux
0,CnMDH,2.7,0.256721,0.767596
1,BmMDH*,3.0,0.231049,0.690837
2,BsMDH,3.0,0.231049,0.690837
3,CgMDH,10.0,0.069315,0.207251


In [27]:
# While the maximum growth the strain can reach is as with foramte
with model_C1 as m:
    m.reactions.EX_glc__D_e.lower_bound = -10.5
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.objective = 'MeDH'
    m.objective_direction = 'min'
    for i in range(len(growth_MDH_for)):
        gr = growth_MDH_for.loc[i,'growth_rate']
        m.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (gr, gr)
        cobra.flux_analysis.pfba(m)
        growth_MDH_for.loc[i,'MeDH_flux'] = m.reactions.MeDH.flux  # in mmol/gCDW/h

growth_MDH_for

,MDH,doubling_time,growth_rate,MeDH_flux
0,CnMDH,2.2,0.315067,0.942050
1,BmMDH*,1.7,0.407734,1.219124
2,BsMDH,2.1,0.330070,0.986910
3,CgMDH,1.6,0.433217,1.295319
